In [ ]:
!pip install mediapipe opencv-python

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from numpy import loadtxt
import os
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
points = mp_pose.PoseLandmark # Landmarks
path = "dataset/good/2" # enter dataset path
data1 = []
for p in points:
        x = str(p)[13:]
        data1.append(x + "_x")
        data1.append(x + "_y")
        data1.append(x + "_z")
        data1.append(x + "_vis")
data2 = []
for p in points:
        x = str(p)[13:]
        data2.append(x + "_x")
        data2.append(x + "_y")
        data2.append(x + "_z")
        data2.append(x + "_vis")
trained1 = []
trained2 = []

In [ ]:
trained1 = loadtxt('good-posture1.txt')
min_angle1 = int(min(trained1))
max_angle1 = int(max(trained1))
trained2 = loadtxt('good-posture2.txt')
min_angle2 = int(min(trained2))
max_angle2 = int(max(trained2))

In [ ]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [ ]:
count = 0
cap = cv2.VideoCapture(1)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            # Calculate angle
            angle1 = calculate_angle(right_shoulder,right_hip,left_hip)
            
            # Visualize angle
            cv2.putText(image, str(angle1), 
                           tuple(np.multiply(left_hip, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            data1[count] = angle1
                 
            if int(data1[count]) in range(min_angle1, max_angle1):
                    
                # Calculate angle
                angle2 = calculate_angle(right_hip,right_shoulder,left_shoulder)
            
                # Visualize angle
                cv2.putText(image, str(angle2), 
                               tuple(np.multiply(left_shoulder, [640, 480]).astype(int)), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                    )
                
                data2[count] = angle2
                
                if int(data2[count]) in range(min_angle2, max_angle2):
                    
                    print("Good Posture")
                    
                else:
                    
                    print("Bad Posture")
                
            else:
                    
                print("Bad Posture")
                    
            count += 1
                       
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()